Iterative Solvers
===

So far we have used direct solvers to solve the linear system of equations. Although a direct solver can profit from the sparse matrix, it's arithmetic complexity is sub-optimal. For large-scale problems iterative solvers are a must.

The conjugate gradient (cg) method is the standard method for symmetric and positive definite matrices. It's convergence rate depends on a preconditioner, what is a cheap approximative inverse to the matrix.

In [14]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo

We generate a 3D geometry and mesh using the OCC constructive solid geometry (CSG) modeler:

In [15]:
from netgen.occ import *
cube = Box((0,0,0),(1,1,1))
cyl = Cylinder((0,0.5,0.5),X, r=0.2, h=1)
cube.faces.name = "outer"
cyl.faces.name = "cyl"
shape = cube-cyl
DrawGeo(shape);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

In [17]:
ngmesh = OCCGeometry(shape).GenerateMesh(maxh=0.1)
for l in range(0):
    ngmesh.Refine()
mesh = Mesh(ngmesh)
mesh.Curve(3)
Draw (mesh);

Highest entry in topology hierarchy: 
1 solid(s)
 set global mesh
 Face 1 / 7 (parameter space projection)
 Surface meshing done
 0 illegal triangles
 Face 2 / 7 (parameter space projection)
 Surface meshing done
 0 illegal triangles
 Face 3 / 7 (parameter space projection)
 Surface meshing done
 0 illegal triangles
 Face 4 / 7 (parameter space projection)
 Surface meshing done
 0 illegal triangles
 Face 5 / 7 (parameter space projection)
 Surface meshing done
 0 illegal triangles
 Face 6 / 7 (parameter space projection)
 Surface meshing done
 0 illegal triangles
 Face 7 / 7 (parameter space projection)
 Surface meshing done
 0 illegal triangles
 Optimization step 0
 Edgeswapping, topological
 Smoothing
 Combine improve
 Smoothing
 Edgeswapping, metric
 Smoothing
 Combine improve
 Smoothing
 Edgeswapping, metric
 Smoothing
 Combine improve
 Smoothing
 Optimization step 0
 Edgeswapping, topological
 Smoothing
 Combine improve
 Smoothing
 Edgeswapping, metric
 Smoothing
 Combine improve


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

In [18]:
fes = H1(mesh, order=3, dirichlet="outer", wb_withoutedges=True)
print ("we have", fes.ndof, "unknowns")
u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm(fes)
a += grad(u)*grad(v)*dx

f = LinearForm(fes)
f += v*dx

# c = Preconditioner(a, "direct", inverse="sparsecholesky")
c = Preconditioner(a, "local")
# c = Preconditioner(a, "bddc")

gfu = GridFunction(fes)

we have 25575 unknowns


assemble system and setup preconditioner in parallel:

In [19]:
ngsglobals.msg_level=5
with TaskManager():
    a.Assemble()
    f.Assemble()

assemble VOL element 4833/4833                                 
assemble VOL element 4833/4833                                 


solve the system using the preconditioned conjugate gradient method:

In [20]:
from ngsolve.krylovspace import CGSolver

with TaskManager():
    inv = CGSolver(mat=a.mat, pre=c.mat, printrates=True, maxiter=400)
    gfu.vec.data = inv * f.vec

CG iteration 1, residual = 0.04653198884871723     
CG iteration 2, residual = 0.05290163957022697     
CG iteration 3, residual = 0.05573659220484592     
CG iteration 4, residual = 0.04548134207412587     
CG iteration 5, residual = 0.03057360298928957     
CG iteration 6, residual = 0.02444369204273861     
CG iteration 7, residual = 0.019620412344585666     
CG iteration 8, residual = 0.014013388449121174     
CG iteration 9, residual = 0.009951850338604361     
CG iteration 10, residual = 0.0077815585449014695     
CG iteration 11, residual = 0.006915536818293857     
CG iteration 12, residual = 0.006452419556672611     
CG iteration 13, residual = 0.00660768482216925     
CG iteration 14, residual = 0.0071333821715744775     
CG iteration 15, residual = 0.00693951716546615     
CG iteration 16, residual = 0.006009073401378995     
CG iteration 17, residual = 0.0047473102005996535     
CG iteration 18, residual = 0.0035891514262892713     
CG iteration 19, residual = 0.00262265874

In [13]:
Draw (gfu);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…